In [1]:
import numpy as np

from tqdm import tqdm
from datasets.video_dataset import VideoDataset
from torch.utils.data import DataLoader

ds = VideoDataset('./data/val', num_frames=22)
loader = DataLoader(ds, batch_size=128)

In [2]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
model = torch.load('model.pt').to(device)

In [4]:
preds = []
masks = []

model.eval()
with torch.inference_mode():
    for batch in tqdm(loader):
        image = batch['images'][:, 11].float().to(device)
        mask = batch['masks'][:, -1]
        
        output = model(image)
        output = torch.argmax(output, dim=1)
        preds.append(output.cpu())
        masks.append(mask)

100%|██████████| 8/8 [01:38<00:00, 12.26s/it]


In [5]:
pred = np.concatenate(preds, axis=0)
mask = np.concatenate(masks, axis=0)

In [6]:
pred.shape

(1000, 160, 240)

In [7]:
mask.shape

(1000, 160, 240)

In [8]:
import torchmetrics

jaccard = torchmetrics.JaccardIndex(task='multiclass', num_classes=49)
jaccard(torch.tensor(pred), torch.tensor(mask))

tensor(0.1845)